In [79]:
from virusPropagationModel import ModeledPopulatedWorld,Simulation
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
 

ImportError: cannot import name 'ModeledPopulatedWorld' from 'virusPropagationModel' (/Users/aviv/PycharmProjects/Corona_model_local/virusPropagationModel.py)

In [ ]:
modeledWorld1 = ModeledPopulatedWorld(100,400,5)

In [ ]:
simulation1 = Simulation(modeledWorld1,100)


In [ ]:
simulation1.plot_status_timecourse()


In [ ]:
# todo: plot location ID/type timecourse.
#res[res['h_ID']==1].plot('time','loc')
#plt.title('schedule')

